Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи
- построить свёрточные архитектуры
- построить различные архитектуры с RNN
- построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [40]:
import re
import os
import nltk
import numpy as np
import pandas as pd
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten, SimpleRNN, LSTM, GRU, Reshape
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [41]:
data_path = r'/home/dmitriy/Downloads/ai_nlp_data/hw_8/'

In [42]:
data = pd.read_excel(os.path.join(data_path, r'отзывы за лето.xls'))

In [43]:
max_words = 20000
max_len = 150
num_classes = 5
epochs = 10
batch_size = 512
print_batch_n = 100

In [44]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dmitriy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/dmitriy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'
morpher = MorphAnalyzer()


def words_only(text):
    text=str(text)
    return text.lower()   


def remove_punkt(text):
    return re.sub(ptrn, ' ', text)


def to_token(text):
    return nltk.tokenize.word_tokenize(text)


def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)


def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)


def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [46]:
data['normalized_content'] = data['Content'].apply(normalize_text)

In [47]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [48]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [49]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]


data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']], 
    dtype=np.int32)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)
le = LabelEncoder()
y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

CNN

In [51]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 12s 478ms/step - loss: 1.2540 - accuracy: 0.6762 - val_loss: 1.2528 - val_accuracy: 0.7155
Epoch 2/10
23/23 [==============================] - 10s 450ms/step - loss: 0.9661 - accuracy: 0.7046 - val_loss: 0.8785 - val_accuracy: 0.7172
Epoch 3/10
23/23 [==============================] - 10s 434ms/step - loss: 0.7606 - accuracy: 0.7242 - val_loss: 0.7998 - val_accuracy: 0.7131
Epoch 4/10
23/23 [==============================] - 10s 425ms/step - loss: 0.6909 - accuracy: 0.7527 - val_loss: 0.7620 - val_accuracy: 0.7463
Epoch 5/10
23/23 [==============================] - 10s 432ms/step - loss: 0.6510 - accuracy: 0.7626 - val_loss: 0.7318 - val_accuracy: 0.7618
Epoch 6/10
23/23 [==============================] - 10s 431ms/step - loss: 0.6217 - accuracy: 0.7838 - val_loss: 0.7261 - val_accuracy: 0.7632
Epoch 7/10
23/23 [==============================] - 10s 428ms/step - loss: 0.5932 - accuracy: 0.7892 - val_loss: 0.7119 - val_accuracy: 0.7646

In [52]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.DataFrame(columns=['model', 'Test score', 'Test accuracy'])
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['CNN'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

 1/13 [=>............................] - ETA: 1s - loss: 0.7886 - accuracy: 0.7305

13/13 [==============================] - 1s 82ms/step - loss: 0.7741 - accuracy: 0.7283
Test score: 0.7740876078605652
Test accuracy: 0.7282994389533997


SimpleRNN

In [53]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 8s 270ms/step - loss: 1.3257 - accuracy: 0.6789 - val_loss: 0.8988 - val_accuracy: 0.7155
Epoch 2/10
23/23 [==============================] - 6s 259ms/step - loss: 0.7918 - accuracy: 0.7114 - val_loss: 0.7137 - val_accuracy: 0.7425
Epoch 3/10
23/23 [==============================] - 6s 262ms/step - loss: 0.6703 - accuracy: 0.7597 - val_loss: 0.6782 - val_accuracy: 0.7715
Epoch 4/10
23/23 [==============================] - 7s 310ms/step - loss: 0.6146 - accuracy: 0.7810 - val_loss: 0.6686 - val_accuracy: 0.7812
Epoch 5/10
23/23 [==============================] - 6s 245ms/step - loss: 0.5647 - accuracy: 0.7950 - val_loss: 0.6708 - val_accuracy: 0.7822
Epoch 6/10
23/23 [==============================] - 6s 259ms/step - loss: 0.5102 - accuracy: 0.8202 - val_loss: 0.6976 - val_accuracy: 0.7812
Epoch 7/10
23/23 [==============================] - 6s 248ms/step - loss: 0.4405 - accuracy: 0.8539 - val_loss: 0.7386 - val_accuracy: 0.7770
Epoch 

In [54]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['SimpleRNN'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 76ms/step - loss: 0.9805 - accuracy: 0.7352
Test score: 0.9805441498756409
Test accuracy: 0.735237181186676


LSTM

In [55]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
 2/23 [=>............................] - ETA: 10s - loss: 1.6087 - accuracy: 0.4180 

23/23 [==============================] - 17s 586ms/step - loss: 1.5655 - accuracy: 0.6783 - val_loss: 1.4617 - val_accuracy: 0.7155
Epoch 2/10
23/23 [==============================] - 13s 556ms/step - loss: 1.2204 - accuracy: 0.7030 - val_loss: 0.9662 - val_accuracy: 0.7155
Epoch 3/10
23/23 [==============================] - 13s 547ms/step - loss: 0.8506 - accuracy: 0.7033 - val_loss: 0.7960 - val_accuracy: 0.7356
Epoch 4/10
23/23 [==============================] - 13s 551ms/step - loss: 0.7293 - accuracy: 0.7633 - val_loss: 0.7439 - val_accuracy: 0.7736
Epoch 5/10
23/23 [==============================] - 15s 663ms/step - loss: 0.6699 - accuracy: 0.7847 - val_loss: 0.7235 - val_accuracy: 0.7770
Epoch 6/10
23/23 [==============================] - 16s 701ms/step - loss: 0.6325 - accuracy: 0.7904 - val_loss: 0.7072 - val_accuracy: 0.7767
Epoch 7/10
23/23 [==============================] - 15s 645ms/step - loss: 0.5981 - accuracy: 0.8001 - val_loss: 0.7183 - val_accuracy: 0.7788
Epoch 8/10

In [56]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['LSTM'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 195ms/step - loss: 0.7613 - accuracy: 0.7509
Test score: 0.7613303661346436
Test accuracy: 0.750887393951416


GRU

In [57]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 17s 564ms/step - loss: 1.5168 - accuracy: 0.6749 - val_loss: 1.2947 - val_accuracy: 0.7155
Epoch 2/10
23/23 [==============================] - 12s 521ms/step - loss: 1.0397 - accuracy: 0.7030 - val_loss: 0.8704 - val_accuracy: 0.7155
Epoch 3/10
23/23 [==============================] - 12s 523ms/step - loss: 0.7807 - accuracy: 0.7031 - val_loss: 0.7220 - val_accuracy: 0.7290
Epoch 4/10
23/23 [==============================] - 12s 526ms/step - loss: 0.6608 - accuracy: 0.7675 - val_loss: 0.6852 - val_accuracy: 0.7708
Epoch 5/10
23/23 [==============================] - 12s 530ms/step - loss: 0.6118 - accuracy: 0.7856 - val_loss: 0.6864 - val_accuracy: 0.7791
Epoch 6/10
23/23 [==============================] - 12s 528ms/step - loss: 0.5783 - accuracy: 0.7955 - val_loss: 0.6934 - val_accuracy: 0.7719
Epoch 7/10
23/23 [==============================] - 12s 530ms/step - loss: 0.5468 - accuracy: 0.8076 - val_loss: 0.7225 - val_accuracy: 0.7722

In [58]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['GRU'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 150ms/step - loss: 0.8087 - accuracy: 0.7491
Test score: 0.808746337890625
Test accuracy: 0.749112606048584


CNN -> LSTM

In [59]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 19s 781ms/step - loss: 1.1732 - accuracy: 0.6741 - val_loss: 0.8876 - val_accuracy: 0.7155
Epoch 2/10
23/23 [==============================] - 18s 792ms/step - loss: 0.8087 - accuracy: 0.7312 - val_loss: 0.7180 - val_accuracy: 0.7573
Epoch 3/10
23/23 [==============================] - 18s 785ms/step - loss: 0.6942 - accuracy: 0.7618 - val_loss: 0.6679 - val_accuracy: 0.7701
Epoch 4/10
23/23 [==============================] - 18s 774ms/step - loss: 0.6258 - accuracy: 0.7805 - val_loss: 0.6547 - val_accuracy: 0.7774
Epoch 5/10
23/23 [==============================] - 18s 775ms/step - loss: 0.5736 - accuracy: 0.7945 - val_loss: 0.6566 - val_accuracy: 0.7736
Epoch 6/10
23/23 [==============================] - 18s 775ms/step - loss: 0.5304 - accuracy: 0.8129 - val_loss: 0.6653 - val_accuracy: 0.7691
Epoch 7/10
23/23 [==============================] - 18s 780ms/step - loss: 0.4837 - accuracy: 0.8320 - val_loss: 0.6979 - val_accuracy: 0.7712

In [60]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['CNN -> LSTM'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 186ms/step - loss: 0.7966 - accuracy: 0.7460
Test score: 0.7965673804283142
Test accuracy: 0.7460471391677856


CNN -> GRU

In [61]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
 5/23 [=====>........................] - ETA: 9s - loss: 1.5730 - accuracy: 0.5820 

23/23 [==============================] - 16s 658ms/step - loss: 1.1976 - accuracy: 0.6748 - val_loss: 0.8108 - val_accuracy: 0.7155
Epoch 2/10
23/23 [==============================] - 16s 718ms/step - loss: 0.7976 - accuracy: 0.7033 - val_loss: 0.7338 - val_accuracy: 0.7221
Epoch 3/10
23/23 [==============================] - 15s 670ms/step - loss: 0.7151 - accuracy: 0.7417 - val_loss: 0.6897 - val_accuracy: 0.7611
Epoch 4/10
23/23 [==============================] - 15s 664ms/step - loss: 0.6403 - accuracy: 0.7758 - val_loss: 0.6633 - val_accuracy: 0.7656
Epoch 5/10
23/23 [==============================] - 15s 671ms/step - loss: 0.5771 - accuracy: 0.7956 - val_loss: 0.6742 - val_accuracy: 0.7674
Epoch 6/10
23/23 [==============================] - 15s 670ms/step - loss: 0.5325 - accuracy: 0.8105 - val_loss: 0.7034 - val_accuracy: 0.7639
Epoch 7/10
23/23 [==============================] - 15s 663ms/step - loss: 0.4960 - accuracy: 0.8249 - val_loss: 0.7241 - val_accuracy: 0.7587
Epoch 8/10

In [62]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['CNN -> GRU'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 188ms/step - loss: 0.8226 - accuracy: 0.7422
Test score: 0.8225601315498352
Test accuracy: 0.7421749234199524


CNN -> SimpleRNN

In [63]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
 9/23 [==========>...................] - ETA: 5s - loss: 1.5440 - accuracy: 0.3123

23/23 [==============================] - 11s 433ms/step - loss: 1.2696 - accuracy: 0.5504 - val_loss: 0.9163 - val_accuracy: 0.7280
Epoch 2/10
23/23 [==============================] - 9s 405ms/step - loss: 0.8889 - accuracy: 0.7223 - val_loss: 0.7991 - val_accuracy: 0.7394
Epoch 3/10
23/23 [==============================] - 9s 397ms/step - loss: 0.7690 - accuracy: 0.7503 - val_loss: 0.7274 - val_accuracy: 0.7594
Epoch 4/10
23/23 [==============================] - 9s 390ms/step - loss: 0.6801 - accuracy: 0.7695 - val_loss: 0.6847 - val_accuracy: 0.7663
Epoch 5/10
23/23 [==============================] - 9s 387ms/step - loss: 0.6165 - accuracy: 0.7904 - val_loss: 0.6720 - val_accuracy: 0.7701
Epoch 6/10
23/23 [==============================] - 10s 422ms/step - loss: 0.5662 - accuracy: 0.8049 - val_loss: 0.6781 - val_accuracy: 0.7743
Epoch 7/10
23/23 [==============================] - 9s 400ms/step - loss: 0.5205 - accuracy: 0.8172 - val_loss: 0.6640 - val_accuracy: 0.7750
Epoch 8/10
23/2

In [64]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['CNN -> SimpleRNN'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 109ms/step - loss: 0.7766 - accuracy: 0.7498
Test score: 0.7766216397285461
Test accuracy: 0.7497580051422119


SimpleRNN -> CNN

In [65]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 15s 616ms/step - loss: 1.2615 - accuracy: 0.6833 - val_loss: 0.8706 - val_accuracy: 0.7214
Epoch 2/10
23/23 [==============================] - 14s 611ms/step - loss: 0.7715 - accuracy: 0.7398 - val_loss: 0.7070 - val_accuracy: 0.7584
Epoch 3/10
23/23 [==============================] - 14s 597ms/step - loss: 0.6440 - accuracy: 0.7688 - val_loss: 0.6758 - val_accuracy: 0.7681
Epoch 4/10
23/23 [==============================] - 14s 616ms/step - loss: 0.5796 - accuracy: 0.7905 - val_loss: 0.7154 - val_accuracy: 0.7684
Epoch 5/10
23/23 [==============================] - 13s 578ms/step - loss: 0.5358 - accuracy: 0.8026 - val_loss: 0.7344 - val_accuracy: 0.7615
Epoch 6/10
23/23 [==============================] - 12s 538ms/step - loss: 0.4931 - accuracy: 0.8224 - val_loss: 0.7974 - val_accuracy: 0.7598
Epoch 7/10
23/23 [==============================] - 12s 536ms/step - loss: 0.4560 - accuracy: 0.8320 - val_loss: 0.8708 - val_accuracy: 0.7560

In [66]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['SimpleRNN -> CNN'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 145ms/step - loss: 1.1344 - accuracy: 0.7247
Test score: 1.1343705654144287
Test accuracy: 0.7247499227523804


LSTM -> CNN

In [67]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 25s 983ms/step - loss: 1.2879 - accuracy: 0.6771 - val_loss: 1.0435 - val_accuracy: 0.7159
Epoch 2/10
23/23 [==============================] - 22s 968ms/step - loss: 0.9297 - accuracy: 0.7327 - val_loss: 0.8308 - val_accuracy: 0.7567
Epoch 3/10
23/23 [==============================] - 25s 1s/step - loss: 0.7578 - accuracy: 0.7648 - val_loss: 0.7586 - val_accuracy: 0.7653
Epoch 4/10
23/23 [==============================] - 22s 977ms/step - loss: 0.6821 - accuracy: 0.7801 - val_loss: 0.7525 - val_accuracy: 0.7698
Epoch 5/10
23/23 [==============================] - 21s 920ms/step - loss: 0.6282 - accuracy: 0.7921 - val_loss: 0.7637 - val_accuracy: 0.7739
Epoch 6/10
23/23 [==============================] - 24s 1s/step - loss: 0.5842 - accuracy: 0.8026 - val_loss: 0.7568 - val_accuracy: 0.7712
Epoch 7/10
23/23 [==============================] - 25s 1s/step - loss: 0.5524 - accuracy: 0.8108 - val_loss: 0.7525 - val_accuracy: 0.7708
Epoch 8/

In [68]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['LSTM -> CNN'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 241ms/step - loss: 0.8934 - accuracy: 0.7460
Test score: 0.8934483528137207
Test accuracy: 0.7460471391677856


GRU -> CNN

In [69]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Epoch 1/10


23/23 [==============================] - 23s 861ms/step - loss: 1.2376 - accuracy: 0.6668 - val_loss: 0.9101 - val_accuracy: 0.7273
Epoch 2/10
23/23 [==============================] - 19s 816ms/step - loss: 0.8672 - accuracy: 0.7308 - val_loss: 0.7740 - val_accuracy: 0.7532
Epoch 3/10
23/23 [==============================] - 19s 826ms/step - loss: 0.7098 - accuracy: 0.7677 - val_loss: 0.7134 - val_accuracy: 0.7684
Epoch 4/10
23/23 [==============================] - 19s 823ms/step - loss: 0.6071 - accuracy: 0.7901 - val_loss: 0.7095 - val_accuracy: 0.7701
Epoch 5/10
23/23 [==============================] - 20s 875ms/step - loss: 0.5410 - accuracy: 0.8026 - val_loss: 0.7164 - val_accuracy: 0.7722
Epoch 6/10
23/23 [==============================] - 19s 819ms/step - loss: 0.4949 - accuracy: 0.8122 - val_loss: 0.7756 - val_accuracy: 0.7688
Epoch 7/10
23/23 [==============================] - 19s 818ms/step - loss: 0.4620 - accuracy: 0.8288 - val_loss: 0.7661 - val_accuracy: 0.7643
Epoch 8/10

In [70]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
metrics_df = pd.concat([metrics_df, pd.DataFrame({
    'model': ['GRU -> CNN'],
    'Test score': [score[0]],
    'Test accuracy': [score[1]],
})])
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 205ms/step - loss: 0.9782 - accuracy: 0.7377
Test score: 0.9781825542449951
Test accuracy: 0.7376573085784912


Итоги

In [71]:
metrics_df.sort_values(by='Test score', ascending=False)

,model,Test score,Test accuracy
0,SimpleRNN -> CNN,1.134371,0.724750
0,SimpleRNN,0.980544,0.735237
0,GRU -> CNN,0.978183,0.737657
0,LSTM -> CNN,0.893448,0.746047
0,CNN -> GRU,0.822560,0.742175
0,GRU,0.808746,0.749113
0,CNN -> LSTM,0.796567,0.746047
0,CNN -> SimpleRNN,0.776622,0.749758
0,CNN,0.774088,0.728299
0,LSTM,0.761330,0.750887


In [72]:
metrics_df.sort_values(by='Test accuracy', ascending=False)

,model,Test score,Test accuracy
0,LSTM,0.761330,0.750887
0,CNN -> SimpleRNN,0.776622,0.749758
0,GRU,0.808746,0.749113
0,CNN -> LSTM,0.796567,0.746047
0,LSTM -> CNN,0.893448,0.746047
0,CNN -> GRU,0.822560,0.742175
0,GRU -> CNN,0.978183,0.737657
0,SimpleRNN,0.980544,0.735237
0,CNN,0.774088,0.728299
0,SimpleRNN -> CNN,1.134371,0.724750
